In [1]:
# !pip install openpyxl

# Imports

In [2]:
import pandas as pd
import openpyxl 

# Data

In [5]:
dir_path = "C:/Users/CPL17/OneDrive/Desktop/"
full_path = dir_path + "PA Wildflower Database.xlsx"

In [6]:
online_long = pd.read_excel(full_path,sheet_name="ONLINE")


In [7]:
metadata= pd.read_excel(full_path,sheet_name="ERA_PA")

In [8]:
online_long.shape

(4481, 4)

In [9]:
online_long.head()

,USDA,Scientific Name,Root,Web
0,ACMI2,Achillea millefolium,ErnstSeed.com,https://duckduckgo.com/?q=!ducky+ErnstSeed.com...
1,ACAM,Acorus americanus,ErnstSeed.com,https://duckduckgo.com/?q=!ducky+ErnstSeed.com...
2,AGFO,Agastache foeniculum,ErnstSeed.com,https://duckduckgo.com/?q=!ducky+ErnstSeed.com...
3,AGAL5,Ageratina altissima,ErnstSeed.com,https://duckduckgo.com/?q=!ducky+ErnstSeed.com...
4,ALSU,Alisma subcordatum,ErnstSeed.com,https://duckduckgo.com/?q=!ducky+ErnstSeed.com...


In [11]:
online_long["Root"].value_counts()

PlantMoreNatives.com      1000
IzelPlants.com             998
PraireMoon.com             994
MidAtlanticNatives.com     987
ToadShade.com              284
ErnstSeed.com              218
Name: Root, dtype: int64

In [12]:
online_long["USDA"].value_counts().iloc[[0,-1]]

CALU4    6
ABBA     1
Name: USDA, dtype: int64

# Aggregations

In [14]:
f = lambda x: ', '.join(map(str, set(x)))
online_agg = online_long.groupby("USDA").agg({"Root":[f,len]})


In [15]:
online_agg.reset_index(inplace=True)
online_agg.columns = ["USDA Symbol","Root","COUNT"]
online_agg.head()

,USDA Symbol,Root,COUNT
0,ABBA,PlantMoreNatives.com,1
1,ACAM,"ErnstSeed.com, PlantMoreNatives.com, MidAtlant...",5
2,ACMI2,"ToadShade.com, ErnstSeed.com, PlantMoreNatives...",6
3,ACNE2,"ToadShade.com, PlantMoreNatives.com, MidAtlant...",3
4,ACPA,"IzelPlants.com, PlantMoreNatives.com, PraireMo...",4


In [16]:
(online_agg["USDA Symbol"].value_counts() != 1).sum() #Ensure Properly Aggregate

0

# Join with MetaData

In [18]:
metadata = metadata[["USDA Symbol","Scientific Name","Common Name"]]
metadata.columns = ["USDA Symbol","Scientific Name","Common Name"]

In [19]:
metadata.head(3)

,USDA Symbol,Scientific Name,Common Name
0,ABBA,Abies balsamea,Balsam Fir
1,ACNE2,Acer negundo,Boxelder
2,ACPE,Acer pensylvanicum,Striped Maple


In [20]:
set(online_agg["USDA Symbol"]).difference(set(metadata["USDA Symbol"]))

set()

In [21]:
final = online_agg.merge(metadata,on="USDA Symbol",how="left")
final.head()

,USDA Symbol,Root,COUNT,Scientific Name,Common Name
0,ABBA,PlantMoreNatives.com,1,Abies balsamea,Balsam Fir
1,ACAM,"ErnstSeed.com, PlantMoreNatives.com, MidAtlant...",5,Acorus americanus,Sweetflag
2,ACMI2,"ToadShade.com, ErnstSeed.com, PlantMoreNatives...",6,Achillea millefolium,Common Yarrow
3,ACNE2,"ToadShade.com, PlantMoreNatives.com, MidAtlant...",3,Acer negundo,Boxelder
4,ACPA,"IzelPlants.com, PlantMoreNatives.com, PraireMo...",4,Actaea pachypoda,White Baneberry


In [22]:
final["Common Name"] = final["Common Name"].str.title()

In [23]:
cond = final["USDA Symbol"].isin(['CETE', 'GABI2', 'GALO3'])
final[cond]

,USDA Symbol,Root,COUNT,Scientific Name,Common Name
167,CETE,"IzelPlants.com, PlantMoreNatives.com, PraireMo...",4,Celtis tenuifolia,Dwarf Hackberry
343,GABI2,"ToadShade.com, ErnstSeed.com, PlantMoreNatives...",6,Gaura biennis,Biennial Beeblossom
348,GALO3,"IzelPlants.com, PlantMoreNatives.com, PraireMo...",4,Gaura longiflora,Longflower Beeblossom


In [24]:
final.loc[cond,"Scientific Name"] = ["Celtis tenuifolia","Gaura biennis L.","Gaura longiflora"]

In [25]:
final["String"] = [f"{row['Common Name']} ({row['Scientific Name']}): {row['Root']}" for _,row in final.iterrows()]


In [26]:
final.head()

,USDA Symbol,Root,COUNT,Scientific Name,Common Name,String
0,ABBA,PlantMoreNatives.com,1,Abies balsamea,Balsam Fir,Balsam Fir (Abies balsamea): PlantMoreNatives.com
1,ACAM,"ErnstSeed.com, PlantMoreNatives.com, MidAtlant...",5,Acorus americanus,Sweetflag,"Sweetflag (Acorus americanus): ErnstSeed.com, ..."
2,ACMI2,"ToadShade.com, ErnstSeed.com, PlantMoreNatives...",6,Achillea millefolium,Common Yarrow,Common Yarrow (Achillea millefolium): ToadShad...
3,ACNE2,"ToadShade.com, PlantMoreNatives.com, MidAtlant...",3,Acer negundo,Boxelder,"Boxelder (Acer negundo): ToadShade.com, PlantM..."
4,ACPA,"IzelPlants.com, PlantMoreNatives.com, PraireMo...",4,Actaea pachypoda,White Baneberry,White Baneberry (Actaea pachypoda): IzelPlants...


In [30]:
final.to_excel("ONLINE_AGG.xlsx")